In [36]:
import json
from pathlib import Path
from openai._client import OpenAI

In [37]:
with open("../apikeys/apikeys.json") as f:
    api_conf = json.load(f)

### TTS

In [38]:
def read_and_prepare_text(filepath, max_len=4096):
    parts = []
    buffer = []

    def buffer_size():
        return sum([len(i) for i in buffer])

    with open(filepath) as f:
        for line in f:
            line = line.strip()
            if buffer_size() + len(line) >= max_len:
                parts.append("\n".join(buffer))
                buffer = []
            buffer.append(line)

    if buffer:
        parts.append("\n".join(buffer))
    return parts

In [54]:
def revise_chinese(client, _text):
    _revised = []
    for i, t in enumerate(_text):
        print(f"processing text part {i}..")
        response = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=[
                {"role": "system", "content": "我会发给你一段文字，请将文字中出现的错别字进行改正、对出现的漏字现象进行补全、删除多余的空格。你不需要告诉我错在哪里，也不需要告诉我如何改正，你直接把改好后的文字发出来就可以了。有一些典型的需要纠正的错误我列在这里供你参考：顾容->顾客, 容户->客户, 措群->措辞, 缺三->缺乏, 自茶->自杀, 周国->周围, 车利者->牟利者, 份造->伪造, 次定->决定。"},
                {"role": "user", "content": t},
            ]
        )
        resp_message = response.choices[0].message.dict()
        _revised.append(resp_message["content"])
    return _revised

In [40]:
filepath = "../outputs/influence/ch4/ch4.txt"
text = read_and_prepare_text(filepath, max_len=1024)

In [41]:
print(len(text))

53


In [42]:
client = OpenAI(
    base_url=api_conf["direct"]["api_host"],
    api_key=api_conf["direct"]["api_keys"]["3.5"]
)

In [55]:
# revised_text = text
revised_text = revise_chinese(client, text)

processing text part 0..
processing text part 1..
processing text part 2..
processing text part 3..
processing text part 4..
processing text part 5..
processing text part 6..
processing text part 7..
processing text part 8..
processing text part 9..
processing text part 10..
processing text part 11..
processing text part 12..
processing text part 13..
processing text part 14..
processing text part 15..
processing text part 16..
processing text part 17..
processing text part 18..
processing text part 19..
processing text part 20..
processing text part 21..
processing text part 22..
processing text part 23..
processing text part 24..
processing text part 25..
processing text part 26..
processing text part 27..
processing text part 28..
processing text part 29..
processing text part 30..
processing text part 31..
processing text part 32..
processing text part 33..
processing text part 34..
processing text part 35..
processing text part 36..
processing text part 37..
processing text part 3

In [56]:
revised_text[0]

'4. 社会认同\n\n影响力研究\n\n几年前，北京一家连锁餐厅的经理与研究者合作完成了一项研究，旨在找出在既有效又不花钱的情况下增加某些菜品销量的办法。他们想看看，能不能在不降低菜品价格、不使用更昂贵的食材升级菜谱、不新增更有经验的大厨，也不聘请顾问对菜品的品质大肆宣传的情况下，就能让顾客更频繁地选择特定菜品。他们想通过直接给菜品贴上一个标签就搞定这件事。经理和研究者的确找到了一种效果特别好的标签，不过和之前预想的诸如“本店特色菜”或“厨师今日特别推荐”这样的标签不同，他们只是给该菜品标上了“最受欢迎”的样式。\n\n结果令人印象深刻。每道菜的平均销量都增长了13%~20%。 显然，这些菜因为受欢迎而变得更加受欢迎。值得一提的是，菜品的销量增长，靠的是一种既康价又完全合乎道德 (这些菜品的确是最受欢迎的品目) 的说服手法，它便于执行，只是此前从来没有经理以这么明显的方式提醒顾客。类似的事情也发生在伦敦。当地一家啤酒厂的酒吧与研究者一起进行了一项实验。酒吧入又处的告示牌如实指出，该啤酒厂当周最受欢迎的酒品是波特。统计发现，波特的销量立刻翻了一番。\n\n这样的结果让我很好奇，为什么其他零售南不提供类似的信息呢?在冰激凌店或冻酸奶店，顾客通常可以点完甜品后再自选巧克力粉末、椰子片、饼干渣等配料。既然受欢迎程度的带动力那么强，你或许以为，经理们应该知道贴出海报，说明当月哪些配料或配料组合最受欢迎。很遗憾，他们并没有这么做。其实尤其是对那些不点配料或是只点一种配料的顾客来说，受欢迎度的真实信息会让他们更多地选择加入配料。举个例子，麦当劳的不少门店都提供一种叫“麦旋风”的甜点。当服务员告诉一组顾客：“来份甜点怎么样?麦旋风是最受欢迎的。” 它的销量就激增了5%。当顾客点了麦旋风，店员接着说，“某某口味的配料顾客最喜欢”，那么，顾客购买的额外配料还会再提升48%。值得一提的是，北京的一家连锁餐厅眉州东坡酒楼受此启发，将“最受欢迎菜品”策略纳入了所有的分店，以测量该策略的可行性和影响力度。\n\n线上影响力\n\n奈飞为何要“泄露商机”'

In [57]:
len(revised_text)

53

In [58]:
speed = 1.0
# voice = "alloy"  # male
voice = "nova"  # female
book_name = "influence"
chapter = 4
output_dir = Path(f'../outputs/{book_name}/ch{chapter}')
output_dir.mkdir(parents=True, exist_ok=True)

In [59]:
output_dir

PosixPath('../outputs/influence/ch4')

In [60]:
finished_no = -1

In [61]:
for section_no, content in enumerate(revised_text):
    if section_no <= finished_no:
        continue
    print(f"tts for {section_no} (length: {len(content)})")
    response = client.audio.speech.create(
        model="tts-1",
        speed=speed,
        voice=voice,
        input=content
    )
    response.stream_to_file(output_dir / f"{book_name}_ch{chapter}_part_{section_no:02}_speed{speed}.mp3")
    finished_no = section_no


tts for 0 (length: 865)
tts for 1 (length: 942)
tts for 2 (length: 935)
tts for 3 (length: 916)
tts for 4 (length: 824)
tts for 5 (length: 848)
tts for 6 (length: 814)
tts for 7 (length: 774)
tts for 8 (length: 887)
tts for 9 (length: 775)
tts for 10 (length: 906)
tts for 11 (length: 920)
tts for 12 (length: 1018)
tts for 13 (length: 955)
tts for 14 (length: 688)
tts for 15 (length: 934)
tts for 16 (length: 895)
tts for 17 (length: 878)
tts for 18 (length: 740)
tts for 19 (length: 694)
tts for 20 (length: 905)
tts for 21 (length: 1028)
tts for 22 (length: 1007)
tts for 23 (length: 906)
tts for 24 (length: 912)
tts for 25 (length: 846)
tts for 26 (length: 822)
tts for 27 (length: 916)
tts for 28 (length: 919)
tts for 29 (length: 878)
tts for 30 (length: 943)
tts for 31 (length: 929)
tts for 32 (length: 868)
tts for 33 (length: 964)
tts for 34 (length: 823)
tts for 35 (length: 990)
tts for 36 (length: 955)
tts for 37 (length: 1000)
tts for 38 (length: 883)
tts for 39 (length: 853)
tts fo

In [40]:
!zip -r 影响力_ch2.zip 影响力_ch2_part_*

zsh:1: command not found: zip


In [35]:
!zip -r ../outputs/krv/ch17.zip ../outputs/krv/ch17/

  adding: ../outputs/krv/ch17/ (stored 0%)
  adding: ../outputs/krv/ch17/krv_ch17_part_09_speed0.85.mp3 (deflated 4%)
  adding: ../outputs/krv/ch17/krv_ch17_part_11_speed0.85.mp3 (deflated 4%)
  adding: ../outputs/krv/ch17/krv_ch17_part_00_speed0.85.mp3 (deflated 4%)
  adding: ../outputs/krv/ch17/krv_ch17_part_19_speed0.85.mp3 (deflated 4%)
  adding: ../outputs/krv/ch17/krv_ch17_part_02_speed0.85.mp3 (deflated 4%)
  adding: ../outputs/krv/ch17/krv_ch17_part_01_speed0.85.mp3 (deflated 4%)
  adding: ../outputs/krv/ch17/krv_ch17_part_16_speed0.85.mp3 (deflated 4%)
  adding: ../outputs/krv/ch17/krv_ch17_part_07_speed0.85.mp3 (deflated 4%)
  adding: ../outputs/krv/ch17/krv_ch17_part_13_speed0.85.mp3 (deflated 4%)
  adding: ../outputs/krv/ch17/KRV_ch17_eng.txt (deflated 66%)
  adding: ../outputs/krv/ch17/krv_ch17_part_22_speed0.85.mp3 (deflated 4%)
  adding: ../outputs/krv/ch17/krv_ch17_part_05_speed0.85.mp3 (deflated 4%)
  adding: ../outputs/krv/ch17/krv_ch17_part_08_speed0.85.mp3 (deflated